In [1]:
import torch
import torchvision
import PIL
import pandas as pd
import numpy as np
import matplotlib.pyplot as plta
import seaborn as sns
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import OneHotEncoder
import torchvision.transforms.functional as fn
from torch import nn
import torch.nn.functional as F
import wandb
import torch.optim as optim
from tqdm import tqdm
import sys
# import autoreload
%load_ext autoreload
%autoreload 2
sys.path.append('../scripts/')
from save_model_every_epoch_torch import save_model
import warnings
warnings.filterwarnings("ignore")
from metrics import find_metrics, find_metrics_macro
from save_model_every_epoch_torch import save_model

In [2]:
multiple_gpus = True
if torch.cuda.is_available():
    if torch.cuda.device_count() > 1:
        multiple_gpus = True
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
csv_file = pd.read_csv('../../files/train.csv')
csv_file['updated_paths'] = csv_file['image'].apply(lambda x: '../../files/train_images/' + x)

In [4]:
def split_datasets(csv_file, test_size):
    train, test = train_test_split(csv_file, test_size=test_size, random_state=42)
    train, val = train_test_split(train, test_size=test_size, random_state=42)
    return train, val, test

In [5]:
def encoding_data(csv_data):
    labels = csv_data['species'].values.tolist()
    encoder = OneHotEncoder()
    encoder = encoder.fit(np.array(labels).reshape(-1, 1))
    return encoder

In [6]:
encoder = encoding_data(csv_file)

In [7]:
class DolphinDataset(Dataset):
    def __init__(self, csv_file, encoder, image_size):
        self.csv_file = csv_file
        self.images = csv_file['updated_paths'].values.tolist()
        self.labels = csv_file['species'].values.tolist()
        self.encoder = encoder
        self.encoded_labels = self.encoder.transform(np.array(self.labels).reshape(-1, 1)).toarray()
        self.convert_rgb = torchvision.transforms.Lambda(lambda x: x.repeat(3, 1, 1))
        self.image_size = image_size

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, idx):
        img = torchvision.io.read_file(self.images[idx])
        img = torchvision.io.decode_jpeg(img)
        if img.shape[0] != 3:
            img = self.convert_rgb(img)
        img = torchvision.transforms.functional.resize(img, (self.image_size[0], self.image_size[0]))
        img = img / 255.0
        return img, torch.Tensor(self.encoded_labels[idx])

In [8]:
train, val, test = split_datasets(csv_file, test_size = 0.01)

In [9]:
# incep_model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained = False)

In [10]:
def get_model():
    incep_model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained = False)
    incep_model.fc = nn.Linear(2048, 30)
    return incep_model

In [11]:
categorical_cross_entropy = nn.CrossEntropyLoss()

Without momentum changes

In [12]:
# def train_model(train_dataset, val_dataset, epochs, batch_size, load_weights=None):
#     model = get_model()
#     wandb.init(project='dolphin',
#                config = {
#                    'arch' : 'InceptionV4',
#                })
#     data_pointers = {
#         'train' : train_dataset,
#         'val' : val_dataset,
#     }
#     if load_weights != None:
#         model.load_state_dict(torch.load(load_weights)['model_state_dict'])
#     optimizer = optim.SGD(model.parameters(), lr=0.001)
#     if next(model.parameters()).is_cuda == False:
#         if multiple_gpus == True:
#             model = nn.DataParallel(model)
#         model = model.to(device)
#     for epoch in range(epochs):
#         train_loss= 0.0
#         train_prec, train_rec = 0.0, 0.0
#         val_loss = 0.0
#         val_prec, val_rec = 0.0, 0.0
#         for phase in ['train', 'val']:
#             if phase == 'train':
#                 model.train()
#             else:
#                 model.eval()
#             running_loss, running_precision, running_recall = 0.0, 0.0, 0.0
#             with tqdm(data_pointers[phase], unit='batch') as tepoch:
#                 for imgs, labels in tepoch:
#                     tepoch.set_description(f'Epoch: {epoch}')
#                     imgs = imgs.to(device)
#                     labels = labels.to(device)
#                     optimizer.zero_grad()
#                     with torch.set_grad_enabled(phase == 'train'):
#                         if phase == 'train':
#                             outputs = model(imgs)[0]
#                         else:
#                             outputs = model(imgs)
#                         loss = categorical_cross_entropy(outputs, labels)
#                         prec, rec = find_metrics_macro(outputs, labels, batch_size=batch_size)
#                         if phase == 'train':
#                             loss.backward()
#                             optimizer.step()
#                     running_loss += loss.item()
#                     running_precision += prec.item()
#                     running_recall += rec.item()
#                     tepoch.set_postfix(loss = loss.item(), precision = prec.item(), recall = rec.item())
#             if phase == 'train':
#                 train_loss = running_loss / len(train_dataset)
#                 train_prec = running_precision / len(train_dataset)
#                 train_rec = running_recall / len(train_dataset)
#                 print(f'{phase} Loss: {float(train_loss)}')
#                 print(f'{phase} Precision: {float(train_prec)}')
#                 print(f'{phase} Recall: {float(train_rec)}')
#             else:
#                 val_loss = running_loss / len(val_dataset)
#                 val_prec = running_precision / len(val_dataset)
#                 val_rec = running_recall / len(val_dataset)
#                 print(f'{phase} Loss: {float(val_loss)}')
#                 print(f'{phase} Precision: {float(val_prec)}')
#                 print(f'{phase} Recall: {float(val_rec)}')
#         wandb.log({
#             'train_loss' : train_loss,
#             'val_loss' : val_loss,
#             'train_precision' : train_prec,
#             'train_recall' : train_rec,
#             'val_precision' : val_prec,
#             'val_recall' : val_rec
#         })

In [13]:
# batch_size = 228
# train_dataset = DolphinDataset(train, encoder)
# val_dataset = DolphinDataset(val, encoder)
# train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = 24, prefetch_factor=2)
# val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, num_workers = 24, prefetch_factor=2)
# train_model(train_dataloader, val_dataloader, batch_size=batch_size, epochs=50)

In [14]:
def train_model(train_dataset, val_dataset, epochs, batch_size, weights_path, load_weights=None):
    model = get_model()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.5)
    wandb.init(project='dolphin',
               config = {
                   'arch' : 'InceptionV4',
                   'batch_size' : batch_size,
                   'epochs' : epochs,
                   'optimizer' : type(optimizer).__name__,
                   'learning rate' : optimizer.param_groups[0]['lr'],
                   'momentum' : optimizer.param_groups[0]['momentum'],
                   'loss_function' : categorical_cross_entropy,
                   'classes' : model.fc.out_features,
                   'Data aug' : None
               })
    data_pointers = {
        'train' : train_dataset,
        'val' : val_dataset,
    }
    if load_weights != None:
        model.load_state_dict(torch.load(load_weights)['model_state_dict'])
    if next(model.parameters()).is_cuda == False:
        if multiple_gpus == True:
            model = nn.DataParallel(model)
        model = model.to(device)
    for epoch in range(epochs):
        train_loss= 0.0
        train_prec, train_rec = 0.0, 0.0
        val_loss = 0.0
        val_prec, val_rec = 0.0, 0.0
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            running_loss, running_precision, running_recall = 0.0, 0.0, 0.0
            with tqdm(data_pointers[phase], unit='batch') as tepoch:
                for imgs, labels in tepoch:
                    tepoch.set_description(f'Epoch: {epoch}')
                    imgs = imgs.to(device)
                    labels = labels.to(device)
                    optimizer.zero_grad()
                    with torch.set_grad_enabled(phase == 'train'):
                        if phase == 'train':
                            outputs = model(imgs)[0]
                        else:
                            outputs = model(imgs)
                        loss = categorical_cross_entropy(outputs, labels)
                        prec, rec = find_metrics_macro(outputs, labels, batch_size=batch_size)
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()
                    running_loss += loss.item()
                    running_precision += prec.item()
                    running_recall += rec.item()
                    tepoch.set_postfix(loss = loss.item(), precision = prec.item(), recall = rec.item())
            if phase == 'train':
                train_loss = running_loss / len(train_dataset)
                train_prec = running_precision / len(train_dataset)
                train_rec = running_recall / len(train_dataset)
                print(f'{phase} Loss: {float(train_loss)}')
                print(f'{phase} Precision: {float(train_prec)}')
                print(f'{phase} Recall: {float(train_rec)}')
            else:
                val_loss = running_loss / len(val_dataset)
                val_prec = running_precision / len(val_dataset)
                val_rec = running_recall / len(val_dataset)
                print(f'{phase} Loss: {float(val_loss)}')
                print(f'{phase} Precision: {float(val_prec)}')
                print(f'{phase} Recall: {float(val_rec)}')
        wandb.log({
            'train_loss' : train_loss,
            'val_loss' : val_loss,
            'train_precision' : train_prec,
            'train_recall' : train_rec,
            'val_precision' : val_prec,
            'val_recall' : val_rec
        })
        save_model(model, epoch, optimizer, multiple_gpus, weights_path)

In [15]:
batch_size = 124
train_dataset = DolphinDataset(train, encoder, image_size = (1024, 1024))
val_dataset = DolphinDataset(val, encoder, image_size = (1024, 1024))
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = 22, prefetch_factor=2)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, num_workers = 22, prefetch_factor=2)
train_model(train_dataloader, val_dataloader, batch_size=batch_size, weights_path='../../weights/torch_with_momentum/', epochs=50)

Using cache found in /home/paperspace/.cache/torch/hub/pytorch_vision_v0.10.0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yashchks87. Use `wandb login --relogin` to force relogin


Epoch: 0: 100%|██████████| 404/404 [11:28<00:00,  1.70s/batch, loss=2.7, precision=0.0556, recall=0.0254]  


train Loss: 2.6736125497534724
train Precision: 0.047517704368237
train Recall: 0.025553455827573297


Epoch: 0: 100%|██████████| 5/5 [00:15<00:00,  3.13s/batch, loss=2.56, precision=0.05, recall=0.0133]  


val Loss: 2.5309441089630127
val Precision: 0.061202745884656906
val Recall: 0.02765726950019598
Weight saved for epoch 0.


...

val Loss: 1.2087214589118958
val Precision: 0.2816273391246796
val Recall: 0.26391349732875824
Weight saved for epoch 48.


Epoch: 49: 100%|██████████| 404/404 [11:16<00:00,  1.67s/batch, loss=1.15, precision=0.293, recall=0.251] 


train Loss: 0.8838083217344662
train Precision: 0.33248164556404153
train Recall: 0.33255799760175225


Epoch: 49: 100%|██████████| 5/5 [00:14<00:00,  3.00s/batch, loss=1.3, precision=0.167, recall=0.117]  


val Loss: 0.9552681565284729
val Precision: 0.3149089187383652
val Recall: 0.3010130047798157
Weight saved for epoch 49.


In [16]:
batch_size = 124
train_dataset = DolphinDataset(train, encoder, image_size = (1024, 1024))
val_dataset = DolphinDataset(val, encoder, image_size = (1024, 1024))
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = 22, prefetch_factor=2)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, num_workers = 22, prefetch_factor=2)
train_model(train_dataloader, val_dataloader, batch_size=batch_size, weights_path='../../weights/torch_with_momentum_v2/', epochs=50, load_weights='../../weights/torch_with_momentum/49.pth')

Using cache found in /home/paperspace/.cache/torch/hub/pytorch_vision_v0.10.0


train_loss,█▇▆▆▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
train_precision,▁▁▂▂▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇█████
train_recall,▁▁▂▂▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇██████
val_loss,█▇▆▆▆▅▆▅▅▄▄▄▆▅▃▃▃▃▃▃▃▂▃▃▃▃▂▂▃▂▂▁▁▂▂▂▁▂▁▁
val_precision,▁▁▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▇▆▆▆▇▇▇▇▇█▇▇█
val_recall,▁▁▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▅▆▇▆▆▆▇▇▇▆▇█▇▇█
train_loss,0.88381
train_precision,0.33248
train_recall,0.33256
val_loss,0.95527
val_precision,0.31491


Epoch: 0: 100%|██████████| 404/404 [11:16<00:00,  1.68s/batch, loss=0.765, precision=0.309, recall=0.314]


train Loss: 0.8774054475349955
train Precision: 0.334300935784779
train Recall: 0.3343126504580573


Epoch: 0: 100%|██████████| 5/5 [00:15<00:00,  3.02s/batch, loss=1.41, precision=0.167, recall=0.14]  


...

val Loss: 0.8040023624897004
val Precision: 0.3682542622089386
val Recall: 0.35943596065044403
Weight saved for epoch 48.


Epoch: 49: 100%|██████████| 404/404 [11:18<00:00,  1.68s/batch, loss=0.443, precision=0.401, recall=0.407]


train Loss: 0.5449036578140637
train Precision: 0.4449137911790668
train Recall: 0.44480252656901237


Epoch: 49: 100%|██████████| 5/5 [00:14<00:00,  2.97s/batch, loss=0.894, precision=0.233, recall=0.217]


val Loss: 0.9754506468772888
val Precision: 0.34767518639564515
val Recall: 0.35233902633190156
Weight saved for epoch 49.


In [14]:
batch_size = 256
train_dataset = DolphinDataset(train, encoder)
val_dataset = DolphinDataset(val, encoder)
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = 24, prefetch_factor=2)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, num_workers = 24, prefetch_factor=2)
train_model(train_dataloader, val_dataloader, batch_size=batch_size, weights_path='../../weights/torch_with_momentum_v3/', epochs=250)

Using cache found in /home/ubuntu/.cache/torch/hub/pytorch_vision_v0.10.0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yashchks87. Use `wandb login --relogin` to force relogin


Epoch: 0: 100%|██████████| 196/196 [04:22<00:00,  1.34s/batch, loss=2.8, precision=0.0379, recall=0.0166] 


train Loss: 2.794513634272984
train Precision: 0.041099494988364835
train Recall: 0.02457012413829869


Epoch: 0: 100%|██████████| 2/2 [00:17<00:00,  8.62s/batch, loss=2.56, precision=0.0496, recall=0.0252]


val Loss: 2.608085036277771
val Precision: 0.05098747834563255
val Recall: 0.02587014250457287
Weight saved for epoch 0.


...

train Loss: 0.4073923087241698
train Precision: 0.5198577829465574
train Recall: 0.5331252992761378


Epoch: 196: 100%|██████████| 2/2 [00:17<00:00,  8.98s/batch, loss=0.566, precision=0.471, recall=0.513]


val Loss: 0.6619004905223846
val Precision: 0.4407465308904648
val Recall: 0.4934449940919876
Weight saved for epoch 196.


Epoch: 197:  50%|█████     | 98/196 [02:36<02:36,  1.60s/batch, loss=0.375, precision=0.506, recall=0.505] 


KeyboardInterrupt: 